In [1]:
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_word = ['?','!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_word]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lematized words", words)

pickle.dump(words,open('text.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

61 documents
22 classes ['bahaya_depresi', 'dampak_depresi', 'depresi', 'depresi_kambuh', 'diagnosis_depresi', 'dimana', 'gejala_depresi', 'gejala_depresi_anak', 'gejala_depresi_lansia', 'gejala_depresi_remaja', 'goodbye', 'jenis_depresi', 'lama_efek_depresi', 'mengatasi_depresi', 'obat_depresi', 'pengobatan_depresi', 'penyebab_depresi', 'salam', 'siapa', 'skor_depresi', 'terimakasih', 'tes_mengetahui_depresi_atau_tidak']
98 unique lematized words ['ada', 'adakah', 'aku', 'anak-anak', 'anti', 'apa', 'apakah', 'atau', 'bagaimana', 'berapa', 'berat', 'berbahaya', 'bertemu', 'bye', 'cara', 'dadah', 'dampak', 'dapat', 'dari', 'depresan', 'depresi', 'deprsi', 'diagnosis', 'dialami', 'diatasi', 'dimana', 'disana', 'efek', 'gejala', 'gimana', 'goodbye', 'haii', 'hallo', 'hello', 'hey', 'heyho', 'hi', 'itu', 'jenis', 'jenis-jenis', 'kabar', 'kalangan', 'kambuh', 'kasih', 'ketika', 'khusus', 'kita', 'lagi', 'lama', 'lansia', 'makasih', 'malam', 'masalah', 'membantu', 'mengatasi', 'mengetahui', 

In [8]:
training = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_word = doc[0]

    pattern_word = [lemmatizer.lemmatize(word.lower()) for word in pattern_word]

    for w in words:
        bag.append(1) if w in pattern_word else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

print('Training data complete!')

Training data complete!


In [46]:
model = Sequential()
model.add(Dense(512, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(254, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=400, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("\n")
print("*" * 50)
print("\n Model Created Complete!")

Epoch 1/400
13/13 [==============================] - 0s 2ms/step - loss: 3.0541 - accuracy: 0.1967
Epoch 2/400
13/13 [==============================] - 0s 2ms/step - loss: 2.8482 - accuracy: 0.2459
Epoch 3/400
13/13 [==============================] - 0s 3ms/step - loss: 2.6112 - accuracy: 0.2459
Epoch 4/400
13/13 [==============================] - 0s 3ms/step - loss: 2.4177 - accuracy: 0.2459
Epoch 5/400
13/13 [==============================] - 0s 2ms/step - loss: 2.1023 - accuracy: 0.3934
Epoch 6/400
13/13 [==============================] - 0s 2ms/step - loss: 1.8681 - accuracy: 0.5246
Epoch 7/400
13/13 [==============================] - 0s 2ms/step - loss: 1.5791 - accuracy: 0.5574
Epoch 8/400
13/13 [==============================] - 0s 2ms/step - loss: 1.3894 - accuracy: 0.5082
Epoch 9/400
13/13 [==============================] - 0s 2ms/step - loss: 1.2309 - accuracy: 0.6557
Epoch 10/400
13/13 [==============================] - 0s 2ms/step - loss: 1.0044 - accuracy: 0.7213
Epoch 11/